In [1]:
import numpy as np
import gnss_lib_py as glp
import matplotlib.pyplot as plt
from datetime import datetime, timezone
import pandas as pd

In [2]:
# glp.make_dir("/Users/thomashuang/school/AA 272/data")
# !wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android/measurements/pixel6.txt --quiet -nc -O "/Users/thomashuang/school/AA 272/data/gnss_log.txt"

In [3]:
path = './Pool/gnss_log_2024_11_06_11_13_38.txt'
fix_data = glp.AndroidRawFixes(path)

In [4]:
fix_types = np.unique(fix_data["fix_provider"]).tolist()
fix_types

['fused', 'gnss']

In [5]:
fixes = []
for provider in fix_types:
    fix_provider = fix_data.where("fix_provider",provider)
    fix_provider.rename({"lat_rx_deg":"lat_rx_" + provider + "_deg",
                         "lon_rx_deg":"lon_rx_" + provider + "_deg",
                         "alt_rx_m":"alt_rx_" + provider + "_m",
                         }, inplace=True)
    fixes.append(fix_provider)

In [6]:
fig_fix = glp.plot_map(*fixes)
fig_fix.show()

In [7]:
raw_data = glp.AndroidRawGnss(input_path=path,
                              filter_measurements=True,
                              measurement_filters={"sv_time_uncertainty" : 500.},
                              verbose=True)

sv_time_uncertainty removed 195


In [14]:
# figs = glp.plot_metric_by_constellation(raw_data,"gps_millis","raw_pr_m")

In [9]:
timestamp_start = datetime(year=2024, month=11, day=6, hour=0, tzinfo=timezone.utc)
timestamp_end = datetime(year=2024, month=11, day=6, hour=1, tzinfo=timezone.utc)
gps_millis = glp.datetime_to_gps_millis(np.array([timestamp_start,timestamp_end]))
sp3_path = glp.load_ephemeris(file_type="sp3",
                              gps_millis=gps_millis,
                              verbose=True)

ephemeris dates needed: [datetime.date(2024, 11, 5), datetime.date(2024, 11, 6)]
using previously downloaded file:
 /Users/thomashuang/school/AA 272/final/dgnss_aa272/data/ephemeris/sp3/GFZ0MGXRAP_20243100000_01D_05M_ORB.SP3
using previously downloaded file:
 /Users/thomashuang/school/AA 272/final/dgnss_aa272/data/ephemeris/sp3/GFZ0MGXRAP_20243110000_01D_05M_ORB.SP3
using previously downloaded file:
 /Users/thomashuang/school/AA 272/final/dgnss_aa272/data/ephemeris/sp3/GFZ0MGXRAP_20243100000_01D_05M_ORB.SP3
using previously downloaded file:
 /Users/thomashuang/school/AA 272/final/dgnss_aa272/data/ephemeris/sp3/GFZ0MGXRAP_20243110000_01D_05M_ORB.SP3


In [10]:
# raw_data.where("sv_id", 6, "eq")

In [11]:
# fix_provider = fix_data.where("fix_provider",'gnss')
# fix_provider

In [12]:
# Initialize an empty list to hold the rows of data
data = []

# Open and read the file line by line
with open('/Users/thomashuang/Downloads/2021-11-06_1600_AlloySU_8hr1hz.Obs.txt', 'r') as file:
    for line in file:
        # Process each line
        # Strip whitespace and split by a delimiter (e.g., space, comma, tab)
        # Adjust the delimiter based on your file format
        columns = line.strip().split()  # For space-separated values, use split()
        
        # Append the columns as a row in the data list
        data.append(columns)

# Convert the list of rows into a pandas DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df)

           0           1         2   3    4  5    6  7  8      9   ... 11  12  \
0        2182  576000.000  0.000002  96   18  0   15  0  0  107.0  ...  0   0   
1        2182  576000.000  0.000002  96   18  0   15  0  0  107.0  ...  0  20   
2        2182  576000.000  0.000002  96   18  0   15  0  0  107.0  ...  1   2   
3        2182  576000.000  0.000002  96   18  0   15  0  0  107.0  ...  1   5   
4        2182  576000.000  0.000002  96   18  0   15  0  0  107.0  ...  2   8   
...       ...         ...       ...  ..  ... ..  ... .. ..    ...  ... ..  ..   
4807467  2182  604799.000  0.000002  96  131  0  113  1  0  171.5  ...  2   6   
4807468  2182  604799.000  0.000002  96  138  0  114  1  0  156.5  ...  0   0   
4807469  2182  604799.000  0.000002  96  138  0  114  1  0  156.5  ...  2   6   
4807470  2182  604799.000  0.000002  96  135  0  115  1  0  184.5  ...  0   0   
4807471  2182  604799.000  0.000002  96  135  0  115  1  0  184.5  ...  2   6   

                       13  

In [13]:
sp3 = glp.Sp3(sp3_path)
def find_sv_pos(sp3, sv_id):
    """ finding sv position with ephemeris data FOR DURAND receiver"""
    cropped_sp3_sv = sp3.where("sv_id",sv_id,"eq")
    return cropped_sp3_sv

def get_measured_pseudorange(df,sv_id):
    """ get measured pseudorange of specific sv for durand receiver"""
    filtered_df = df[df.iloc[:,4] == str(sv_id)]
    return filtered_df.iloc[:,14]

def distance_to_groundtruth(sv_pos):
    """ calculate ground truth distance from SV to Durand receiver"""
    GroundTruth = np.array([[37.426906], [-122.173278], [21.010]]) 
    gt_ecef = glp.geodetic_to_ecef(GroundTruth).flatten()
    distance = np.linalg.norm(sv_pos - gt_ecef)
    return distance

def correction(gt_distance, measured_distance, pseudorange):
    """ gt_distance: ground truth distance from distance_to_groundtruth function
    measured_distance: pseudorange for base station
    pseudorange: pseudorange for phone """
    e = gt_distance - measured_distance
    adjusted_pseudorange = pseudorange + e
    return adjusted_pseudorange

def get_pseudorange(raw_data, sv_id):
    """ get measure pseudorange of the rover with specified sv_id"""
    cropped_data = raw_data.where("sv_id", sv_id, "eq")
    pseudorange = cropped_data["raw_pr_m"]
    return pseudorange

In [ ]:
sv_id = 6
sv_pos = find_sv_pos(sp3, sv_id)['x_sv_m', 'y_sv_m', 'z_sv_m'][:,0]
gt_distance = distance_to_groundtruth(sv_pos)
measured_distance = get_measured_pseudorange(df,sv_id)
measured_distance = float(measured_distance.iloc[0])
rover_distance = get_pseudorange(raw_data, sv_id)
# rover_distance = rover_distance[0]
corrected_rover_dis = correction(gt_distance, measured_distance, rover_distance)
corrected_rover_dis

129510382145690.8

In [50]:
GroundTruth = np.array([[37.426906], [-122.173278], [21.010]]) 
gt = glp.geodetic_to_ecef(GroundTruth)

In [47]:
sv_pos

array([-16128724.594,  37694803.653, -10366714.031])

In [52]:
np.linalg.norm(gt-sv_pos)

74548875.14478524

In [28]:
cropped_sp3 = sp3.where("gps_millis",gps_millis[0],"geq").where("gps_millis",gps_millis[1],"leq")


In [49]:
raw_data['raw_pr_m'].min()

129510361629855.75

In [22]:
filtered_df = df[df.iloc[:,4] == str(18)]
filtered_df.iloc[:,14]

0          22053358.82091
1          22053356.74018
2          22053362.54297
3          22053362.51914
4          22053363.77873
                ...      
3379604    24398147.85002
3379774    24398848.88223
3379944    24399551.08854
3380114    24400255.42475
3380284    24400961.73080
Name: 14, Length: 91474, dtype: object